In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/web-page-phishing-detection-dataset/dataset_phishing.csv


In [3]:
df = pd.read_csv("/kaggle/input/web-page-phishing-detection-dataset/dataset_phishing.csv")
print(df.columns)/ kaggle/input/web-page-phishing-detection-dataset/dataset_phishing.csv

Index(['url', 'length_url', 'length_hostname', 'ip', 'nb_dots', 'nb_hyphens',
       'nb_at', 'nb_qm', 'nb_and', 'nb_or', 'nb_eq', 'nb_underscore',
       'nb_tilde', 'nb_percent', 'nb_slash', 'nb_star', 'nb_colon', 'nb_comma',
       'nb_semicolumn', 'nb_dollar', 'nb_space', 'nb_www', 'nb_com',
       'nb_dslash', 'http_in_path', 'https_token', 'ratio_digits_url',
       'ratio_digits_host', 'punycode', 'port', 'tld_in_path',
       'tld_in_subdomain', 'abnormal_subdomain', 'nb_subdomains',
       'prefix_suffix', 'random_domain', 'shortening_service',
       'path_extension', 'nb_redirection', 'nb_external_redirection',
       'length_words_raw', 'char_repeat', 'shortest_words_raw',
       'shortest_word_host', 'shortest_word_path', 'longest_words_raw',
       'longest_word_host', 'longest_word_path', 'avg_words_raw',
       'avg_word_host', 'avg_word_path', 'phish_hints', 'domain_in_brand',
       'brand_in_subdomain', 'brand_in_path', 'suspecious_tld',
       'statistical_report', 

NameError: name 'kaggle' is not defined

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import classification_report, accuracy_score, precision_recall_fscore_support
from xgboost import XGBClassifier
import string
import warnings
warnings.filterwarnings('ignore')

# For LSTM models
try:
    from tensorflow.keras.models import Model, Sequential
    from tensorflow.keras.layers import LSTM, Dense, Dropout, Input, Concatenate, Embedding
    from tensorflow.keras.preprocessing.sequence import pad_sequences
    from tensorflow.keras.callbacks import EarlyStopping
    KERAS_AVAILABLE = True
except ImportError:
    print("⚠️ TensorFlow/Keras not available. Install with: pip install tensorflow")
    KERAS_AVAILABLE = False

# === Load and Prepare Data ===
print("Loading dataset...")
df = pd.read_csv("/kaggle/input/web-page-phishing-detection-dataset/dataset_phishing.csv")

# Keep URL and all features
url_col = df['url'].copy()
status_col = df['status'].copy()

# Map status to binary
status_map = {'legitimate': 0, 'benign': 0, 'phishing': 1}
status_col = status_col.map(status_map)

# Remove rows with unmapped status
valid_idx = status_col.notna()
url_col = url_col[valid_idx]
status_col = status_col[valid_idx]
df_features = df[valid_idx].copy()

print(f"Total samples: {len(status_col)}")
print(f"Phishing: {sum(status_col == 1)}, Legitimate: {sum(status_col == 0)}")

# === Character-Level URL Encoding ===
def encode_url(url, max_len=200):
    """Encode URL as sequence of character indices"""
    charset = string.ascii_letters + string.digits + string.punctuation
    char_to_int = {ch: i + 1 for i, ch in enumerate(charset)}  # start from 1, 0 reserved for padding
    encoded = [char_to_int.get(c, 0) for c in url[:max_len]]
    if len(encoded) < max_len:
        encoded += [0] * (max_len - len(encoded))
    return encoded

print("\nEncoding URLs...")
X_url_encoded = np.array([encode_url(u) for u in url_col])
y = status_col.values.astype(int)

# === Extract Numerical Features ===
# Select only numerical features (exclude 'url' and 'status')
feature_cols = [col for col in df_features.columns if col not in ['url', 'status']]
X_features = df_features[feature_cols].fillna(0).values

print(f"URL encoding shape: {X_url_encoded.shape}")
print(f"Features shape: {X_features.shape}")

# === Train-Test Split ===
# Split all data together to maintain alignment
X_url_train, X_url_test, X_feat_train, X_feat_test, y_train, y_test = train_test_split(
    X_url_encoded, X_features, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTraining samples: {len(y_train)}")
print(f"Testing samples: {len(y_test)}")

# ==========================================
# MODEL 1: Ensemble (Baseline - Your Original)
# ==========================================
print("\n" + "="*60)
print("MODEL 1: Ensemble (RandomForest + XGBoost) on URL Encoding")
print("="*60)

rf = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)

ensemble = VotingClassifier(
    estimators=[('rf', rf), ('xgb', xgb)],
    voting='hard'
)

print("Training ensemble model...")
ensemble.fit(X_url_train, y_train)
y_pred_ensemble = ensemble.predict(X_url_test)

print("\nEnsemble Model Results:")
print(classification_report(y_test, y_pred_ensemble))
ensemble_acc = accuracy_score(y_test, y_pred_ensemble)

# ==========================================
# MODEL 2: LSTM on URL Only
# ==========================================
if KERAS_AVAILABLE:
    print("\n" + "="*60)
    print("MODEL 2: LSTM on URL Character Sequences")
    print("="*60)
    
    # Define LSTM model for URL only
    vocab_size = len(string.ascii_letters + string.digits + string.punctuation) + 1
    max_len = 200
    
    model_lstm_url = Sequential([
        Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len),
        LSTM(128, return_sequences=True, dropout=0.2),
        LSTM(64, dropout=0.2),
        Dense(32, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    
    model_lstm_url.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    print("\nLSTM URL Model Architecture:")
    model_lstm_url.summary()
    
    print("\nTraining LSTM URL model...")
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
    
    history_url = model_lstm_url.fit(
        X_url_train, y_train,
        validation_split=0.2,
        epochs=10,
        batch_size=64,
        callbacks=[early_stop],
        verbose=1
    )
    
    # Predict
    y_pred_lstm_url_prob = model_lstm_url.predict(X_url_test, verbose=0)
    y_pred_lstm_url = (y_pred_lstm_url_prob > 0.5).astype(int).flatten()
    
    print("\nLSTM URL Model Results:")
    print(classification_report(y_test, y_pred_lstm_url))
    lstm_url_acc = accuracy_score(y_test, y_pred_lstm_url)

# ==========================================
# MODEL 3: LSTM on URL + Features (Hybrid)
# ==========================================
if KERAS_AVAILABLE:
    print("\n" + "="*60)
    print("MODEL 3: Hybrid LSTM (URL + Numerical Features)")
    print("="*60)
    
    # Normalize features for better neural network performance
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    X_feat_train_scaled = scaler.fit_transform(X_feat_train)
    X_feat_test_scaled = scaler.transform(X_feat_test)
    
    # Define hybrid model
    # Input 1: URL sequence
    url_input = Input(shape=(max_len,), name='url_input')
    url_embedding = Embedding(input_dim=vocab_size, output_dim=128)(url_input)
    lstm1 = LSTM(128, return_sequences=True, dropout=0.2)(url_embedding)
    lstm2 = LSTM(64, dropout=0.2)(lstm1)
    
    # Input 2: Numerical features
    feature_input = Input(shape=(X_feat_train_scaled.shape[1],), name='feature_input')
    feature_dense = Dense(64, activation='relu')(feature_input)
    feature_dropout = Dropout(0.3)(feature_dense)
    feature_dense2 = Dense(32, activation='relu')(feature_dropout)
    
    # Concatenate both branches
    concatenated = Concatenate()([lstm2, feature_dense2])
    
    # Final layers
    dense1 = Dense(64, activation='relu')(concatenated)
    dropout = Dropout(0.3)(dense1)
    dense2 = Dense(32, activation='relu')(dropout)
    output = Dense(1, activation='sigmoid', name='output')(dense2)
    
    # Create model
    model_hybrid = Model(inputs=[url_input, feature_input], outputs=output)
    
    model_hybrid.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    
    print("\nHybrid LSTM Model Architecture:")
    model_hybrid.summary()
    
    print("\nTraining Hybrid LSTM model...")
    history_hybrid = model_hybrid.fit(
        [X_url_train, X_feat_train_scaled], y_train,
        validation_split=0.2,
        epochs=10,
        batch_size=64,
        callbacks=[early_stop],
        verbose=1
    )
    
    # Predict
    y_pred_hybrid_prob = model_hybrid.predict([X_url_test, X_feat_test_scaled], verbose=0)
    y_pred_hybrid = (y_pred_hybrid_prob > 0.5).astype(int).flatten()
    
    print("\nHybrid LSTM Model Results:")
    print(classification_report(y_test, y_pred_hybrid))
    hybrid_acc = accuracy_score(y_test, y_pred_hybrid)

# ==========================================
# COMPARISON SUMMARY
# ==========================================
print("\n" + "="*60)
print("PERFORMANCE COMPARISON SUMMARY")
print("="*60)

results = []
results.append(("Ensemble (RF + XGBoost)", ensemble_acc))

if KERAS_AVAILABLE:
    results.append(("LSTM on URL Only", lstm_url_acc))
    results.append(("Hybrid LSTM (URL + Features)", hybrid_acc))
    
    # Detailed metrics
    print("\nDetailed Metrics:")
    for name, y_pred in [
        ("Ensemble", y_pred_ensemble),
        ("LSTM URL", y_pred_lstm_url),
        ("Hybrid LSTM", y_pred_hybrid)
    ]:
        precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary')
        print(f"\n{name}:")
        print(f"  Accuracy:  {accuracy_score(y_test, y_pred):.4f}")
        print(f"  Precision: {precision:.4f}")
        print(f"  Recall:    {recall:.4f}")
        print(f"  F1-Score:  {f1:.4f}")

print("\n" + "-"*60)
print("Accuracy Comparison:")
print("-"*60)
results.sort(key=lambda x: x[1], reverse=True)
for i, (model_name, acc) in enumerate(results, 1):
    print(f"{i}. {model_name:35s} {acc:.4f} ({acc*100:.2f}%)")
print("-"*60)

print("\n✅ Analysis complete!")
print("\nKey Takeaways:")
print("1. Ensemble models are strong baselines for structured features")
print("2. LSTM can capture sequential patterns in URL strings")
print("3. Hybrid approach combines both URL patterns and engineered features")
print("4. The best model depends on your deployment requirements (speed vs accuracy)")

2025-11-27 04:18:44.555816: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764217124.872792      38 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764217124.958508      38 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading dataset...
Total samples: 11430
Phishing: 5715, Legitimate: 5715

Encoding URLs...
URL encoding shape: (11430, 200)
Features shape: (11430, 87)

Training samples: 9144
Testing samples: 2286

MODEL 1: Ensemble (RandomForest + XGBoost) on URL Encoding
Training ensemble model...

Ensemble Model Results:
              precision    recall  f1-score   support

           0       0.83      0.89      0.86      1143
           1       0.88      0.82      0.85      1143

    accuracy                           0.86      2286
   macro avg       0.86      0.86      0.86      2286
weighted avg       0.86      0.86      0.86      2286


MODEL 2: LSTM on URL Character Sequences

LSTM URL Model Architecture:


2025-11-27 04:19:02.931544: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

...

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ url_input           │ (None, 200)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ feature_input       │ (None, 87)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 200, 128)  │     12,160 │ url_input[0][0]   │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      5,632 │ feature_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_2 (LSTM)       │ (None, 200, 128)  │    131,584 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_3 (LSTM)       │ (None, 64)        │     49,408 │ lstm_2[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      2,080 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 96)        │          0 │ lstm_3[0][0],     │
│ (Concatenate)       │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 64)        │      6,208 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64)        │          0 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 32)        │      2,080 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output (Dense)      │ (None, 1)         │         33 │ dense_5[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 209,185 (817.13 KB)

 Trainable params: 209,185 (817.13 KB)

 Non-trainable params: 0 (0.00 B)


Training Hybrid LSTM model...
Epoch 1/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 57s 448ms/step - accuracy: 0.7343 - loss: 0.5071 - val_accuracy: 0.9453 - val_loss: 0.1573
Epoch 2/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 50s 437ms/step - accuracy: 0.9301 - loss: 0.1939 - val_accuracy: 0.9535 - val_loss: 0.1377
Epoch 3/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 50s 435ms/step - accuracy: 0.9372 - loss: 0.1669 - val_accuracy: 0.9590 - val_loss: 0.1288
Epoch 4/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 49s 428ms/step - accuracy: 0.9493 - loss: 0.1439 - val_accuracy: 0.9590 - val_loss: 0.1257
Epoch 5/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 49s 430ms/step - accuracy: 0.9575 - loss: 0.1280 - val_accuracy: 0.9595 - val_loss: 0.1213
Epoch 6/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 49s 425ms/step - accuracy: 0.9540 - loss: 0.1286 - val_accuracy: 0.9623 - val_loss: 0.1228
Epoch 7/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 48s 421ms/step - accuracy: 0.9566 - loss: 0.1138 - val_accuracy: 0.9639 - val_loss: 0.1198
Epoch 8/10
115/115 ━━━━━━━━━━━━━━━━━━━━ 48s 418ms/st

In [5]:
if KERAS_AVAILABLE:
    print("\n" + "="*60)
    print("SAVING HYBRID LSTM MODEL")
    print("="*60)
    
    import joblib
    
    # Save just the pickle file with everything needed
    model_hybrid.save('hybrid_lstm_model.h5')
    
    complete_package = {
        'model_path': 'hybrid_lstm_model.h5',
        'scaler': scaler,
        'max_len': max_len,
        'vocab_size': vocab_size,
        'char_to_int': {ch: i + 1 for i, ch in enumerate(string.ascii_letters + string.digits + string.punctuation)},
        'feature_columns': feature_cols
    }
    
    joblib.dump(complete_package, 'hybrid_lstm_complete.pkl')
    print("✅ Saved: hybrid_lstm_complete.pkl")
    print("✅ Saved: hybrid_lstm_model.h5")
else:
    print("\n⚠️ Keras not available - cannot save LSTM models")


SAVING HYBRID LSTM MODEL
✅ Saved: hybrid_lstm_complete.pkl
✅ Saved: hybrid_lstm_model.h5


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
import joblib
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from xgboost import XGBClassifier
import joblib
# === Load and Prepare Data ===
df = pd.read_csv("/kaggle/input/web-page-phishing-detection-dataset/dataset_phishing.csv")

# Drop 'url' column (we assume features are already extracted)
df = df.drop(columns=["url"])

# Encode target
df['status'] = df['status'].map({'legitimate': 0, 'phishing': 1})

X = df.drop("status", axis=1)
y = df["status"]

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === Define Base Models ===
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', verbosity=0)
rf = RandomForestClassifier(n_estimators=100, random_state=42)
lr = LogisticRegression(max_iter=1000)

# === Create Ensemble Model (VotingClassifier) ===
ensemble = VotingClassifier(
    estimators=[('xgb', xgb), ('rf', rf), ('lr', lr)],
    voting='soft'  # Use 'hard' for majority class, 'soft' for probabilities
)

# === Train Ensemble ===
ensemble.fit(X_train, y_train)

# === Evaluate ===
y_pred = ensemble.predict(X_test)
print(classification_report(y_test, y_pred))
# === Save model and column order ===



              precision    recall  f1-score   support

           0       0.97      0.97      0.97      1157
           1       0.97      0.97      0.97      1129

    accuracy                           0.97      2286
   macro avg       0.97      0.97      0.97      2286
weighted avg       0.97      0.97      0.97      2286



In [6]:
!pip install tldextract

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 4.6 MB/s eta 0:00:00


In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report
import joblib
import string

# Load dataset
df = pd.read_csv("/kaggle/input/web-page-phishing-detection-dataset/dataset_phishing.csv")
df = df[['url', 'status']].dropna()

# Map status to binary
df = df[df['status'].isin(['phishing', 'benign'])]
df['status'] = df['status'].map({'phishing': 1, 'benign': 0})

# === Character-Level URL Encoding ===
def encode_url(url, max_len=200):
    charset = string.ascii_letters + string.digits + string.punctuation
    char_to_int = {ch: i + 1 for i, ch in enumerate(charset)}  # start from 1
    encoded = [char_to_int.get(c, 0) for c in url[:max_len]]
    if len(encoded) < max_len:
        encoded += [0] * (max_len - len(encoded))
    return encoded

X = np.array([encode_url(u) for u in df['url']])
y = df['status'].values.astype(int)

# === Train-Test Split ===
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# === Classifiers ===
rf = RandomForestClassifier(n_estimators=100, random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

# === Ensemble ===
ensemble = VotingClassifier(
    estimators=[
        ('rf', rf),
        ('xgb', xgb)
    ],
    voting='hard'  # Switch to 'soft' after verifying predict_proba
)

# === Train & Predict ===
ensemble.fit(X_train, y_train)
y_pred = ensemble.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00      1143

    accuracy                           1.00      1143
   macro avg       1.00      1.00      1.00      1143
weighted avg       1.00      1.00      1.00      1143



In [14]:
joblib.dump(model, "phishing_ensemble_ml_model.pkl")
joblib.dump(X.columns.tolist(), "feature_columns.pkl")

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [18]:
dictt={
    'max_len': 200,
    'char_to_int': {ch: i + 1 for i, ch in enumerate(string.ascii_letters + string.digits + string.punctuation)}
}
joblib.dump(dictt, "feature_columns.pkl")

['feature_columns.pkl']

In [ ]:
model = joblib.load("/kaggle/working/phishing_ensemble_ml_model.pkl")
char2idx = joblib.load("/kaggle/working/url_encoding_metadata.pkl")
maxlen = 200

def encode_url_for_inference(url):
    url = url[:maxlen].ljust(maxlen)
    return [char2idx.get(char, 0) for char in url]

test_url = "http://www.crestonwood.com/router.php"
X_new = np.array([encode_url_for_inference(test_url)])
prediction = model.predict(X_new)
print("Phishing" if prediction[0] == 1 else "Benign")